#### Processamento de Big Data: Projeto Final

Licenciatura em Ciência de Dados

CDB1

Marco Delgado Esperança, Nº 110451

Maria João Ferreira Lourenço, Nº 104716

Docente: João Oliveira

8 de abril de 2023

Flight Status Prediction

https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022

# Análise Exploratória

Esta corresponde à segunda fase do trabalho, onde realizamos um conjunto de estatísticas descritivas acerca de variáveis presentes no ficheiro parquet e as suas relações. É de notar que apenas estudamos as que nos pareceram mais interessantes e relevantes.

In [5]:
spark

In [11]:
# bibliotecas 
import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import desc, count, avg

# Importação do ficheiro parquet

In [12]:
filename = ["./flights_clean.parquet"]

In [13]:
df_flights_clean = spark.read.format("parquet").load(filename)

In [29]:
print("Número de linhas: ", df_flights_clean.count(), "      ||   Número de colunas: ", len(df_flights_clean.columns))

Número de linhas:  28339509       ||   Número de colunas:  47


In [30]:
# number of operating airline tem duas correlações 0.4 e uma alta
cols_to_drop = ["Flight_Number_Operating_Airline", 
                "DOT_ID_Operating_Airline",
                "OriginCityMarketID",
                "OriginStateFips",
                "OriginWac", # 0.5
                "DestCityMarketID",
                "DestStateFips",
                "DestWac",
                "TaxiOut",
                "TaxiIn",
                "CRSArrTime",
                "ArrivalDelayGroups",
                "DistanceGroup",
                "DivAirportLandings"
               ]

In [5]:
df_flights_clean.columns

['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityName', 'OriginState', 'OriginStateName', 'DestAirportID', 'DestAirportSeqID', 'DestCityName', 'DestState', 'DestStateName', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'WheelsOff', 'WheelsOn', 'ArrDelay', 'ArrDel15', 'ArrTimeBlk']

## Descritivas das variáveis numéricas

In [32]:
def describe_numeric_cols_v2(df):
    numeric_cols = [col_name for col_name, col_type in df.dtypes if col_type in ('int', 'bigint', 'float', 'double') and col_name not in cols_to_drop]
    return df.select(numeric_cols).describe().toPandas().transpose()

describe_numeric_cols_v2(df_flights_clean)

23/04/03 19:39:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,0,1,2,3,4
summary,count,mean,stddev,min,max
CRSDepTime,28339509,1325.3256407159347,482.13724257483176,1,2359
DepTime,28339509,1329.1466037396767,495.0205944665375,1.0,2400.0
DepDelayMinutes,28339509,12.717445669224544,45.99462412502477,0.0,7223.0
DepDelay,28339509,9.238519164181708,47.100785715662155,-611.0,7223.0
ArrTime,28339509,1467.9772431131394,525.3427738955158,1.0,2400.0
ArrDelayMinutes,28339509,12.809568401485008,45.80091701166583,0.0,7232.0
AirTime,28339509,109.08957346438147,69.06727016403666,4.0,1557.0
CRSElapsedTime,28339509,138.95247677015152,70.93263349401167,-292.0,1645.0
ActualElapsedTime,28339509,133.32010836885001,70.77018240814341,13.0,1604.0


Todas as colunas têm o mesmo número de linhas, logo não temos valores omissos, como era esperado, uma vez que procedemos à sua limpeza no notebook relacionado com a 'Importação, transformação e limpeza dos dados'. A média e a dispersão das variáveis varia muito consoante a variável que estamos a falar. Os valores máximos são sempre positivos e os valores mínimos, também o tendem a ser com exceção, das variáveis 'DepDelay', 'CRSElapsedTime', 'DepartureDelayGroups' e 'ArrDelay'. É de notar que as variáveis que assumem valores negativos implicam que existiram voos que partiram antes do tempo.

## Variedade de descritivas interessantes

### Número de voos por ano - 'Year'

In [35]:
df_flights_clean.groupBy("Year").agg(F.count("*").alias("Number of flights")).show()

+----+-----------------+
|Year|Number of flights|
+----+-----------------+
|2018|          5578617|
|2019|          7917263|
|2020|          4712844|
|2021|          6185869|
|2022|          3944916|
+----+-----------------+



Observamos que o ano com mais voos foi 2019, em 2020 houve uma queda, que pode estar associada à pandemia, um aumento em 2021, pois houve abertura de fronteiras consoante a prova de teste negativo e vacina e 2022 é o ano com menos voos porque só temos informação até aos mês 7 - julho.

### Número de voos por trimestre - 'Quarter'

In [43]:
df_flights_clean.groupBy("Quarter").agg(F.count("*").alias("Number of flights")).sort('Quarter').show()

+-------+-----------------+
|Quarter|Number of flights|
+-------+-----------------+
|      1|          7510257|
|      2|          7033393|
|      3|          6987160|
|      4|          6808699|
+-------+-----------------+



O primeiro trimestre é o com mais voos, seguido do segundo, terceiro e quarto. Isto pode acontecer, devido à presença do mês de janeiro - férias escolares e passagem de ano - e de março - férias escolares - e é de notar que o carnaval também se realiza neste trimestre. Também é relevante mencionar que 2022 não tem os dados completos, o que se pode refletir nos dados do terceiro e quarto trimestres. 

### Número de voos por mês - 'Month'

In [42]:
df_flights_clean.groupBy("Month").agg(F.count("*").alias("Number of flights")).sort('Month').show()

+-----+-----------------+
|Month|Number of flights|
+-----+-----------------+
|    1|          2612300|
|    2|          2265069|
|    3|          2632888|
|    4|          2356760|
|    5|          2307502|
|    6|          2369131|
|    7|          2635455|
|    8|          2175988|
|    9|          2175717|
|   10|          2301924|
|   11|          2237636|
|   12|          2269139|
+-----+-----------------+



Os meses com mais voos são julho - 7 -, março - 3 - e janeiro - 1, que são meses marcados pelas férias de verão, de primavera e de inverno. O mês com menos voos é setembro. É de notar masi uma vez que não temos informação acerca dos meses 8 a 12 do ano de 2022.

### Número de voos por dia do mês - 'DayofMonth'

In [71]:
df_flights_clean.groupBy("DayofMonth").agg(F.count("*").alias("Number of flights")).sort('DayofMonth').show()

+----------+-----------------+
|DayofMonth|Number of flights|
+----------+-----------------+
|1         |909801           |
|2         |911237           |
|3         |924375           |
|4         |919262           |
|5         |927825           |
|6         |932159           |
|7         |930824           |
|8         |934997           |
|9         |931210           |
|10        |943059           |
|11        |937785           |
|12        |942133           |
|13        |937193           |
|14        |933053           |
|15        |933478           |
|16        |928763           |
|17        |938894           |
|18        |942858           |
|19        |943291           |
|20        |940305           |
+----------+-----------------+
only showing top 20 rows



Dos dias visíveis o dia 1 é o com menos voos e o dia 19 é o com mais voos. Apesar de não ser visível aqui, colocamos a hipótese de que o dia 31 deve ser o com menos voos, porque é um dia que não existe em todos os meses, esta pode ser confirmada no gráfico intitulado de 'Número de voos de partida atrasados por dia do mês' no notebook 'Visualização'.

### Número de voos por dia da semana - 'DayofWeek'

In [46]:
df_flights_clean.groupBy("DayofWeek").agg(F.count("*").alias("Number of flights")).sort('DayofWeek').show()

+---------+-----------------+
|DayofWeek|Number of flights|
+---------+-----------------+
|        1|          4225014|
|        2|          3938863|
|        3|          4004727|
|        4|          4196953|
|        5|          4228383|
|        6|          3618697|
|        7|          4126872|
+---------+-----------------+



O dia da semana com mais voos é quinta e o com menos voos é sábado.

### Top 10 companhias aéreas com mais voos - 'Airline'

In [49]:
airline_counts = df_flights_clean.groupBy('Airline').agg(count('*').alias('count'))

airline_counts.orderBy('count', ascending=False).limit(10).show()

+--------------------+-------+
|             Airline|  count|
+--------------------+-------+
|Southwest Airline...|5292591|
|Delta Air Lines Inc.|3242140|
|SkyWest Airlines ...|3077213|
|American Airlines...|3031275|
|United Air Lines ...|2301368|
|   Republic Airlines|1239828|
|     JetBlue Airways|1073460|
|           Envoy Air|1033979|
|   Endeavor Air Inc.| 974640|
|         Comair Inc.| 922947|
+--------------------+-------+



Aqui podemos observar que a companhia aérea com mais voos é a Southwest Airlines.

### Número de voos pelo top 10 de origens - 'Origin'

In [52]:
df_flights_clean.groupBy('Origin').count().orderBy(desc('count')).show(10)

+------+-------+
|Origin|  count|
+------+-------+
|   ATL|1347020|
|   ORD|1327411|
|   DEN|1137835|
|   DFW|1064833|
|   CLT| 880449|
|   LAX| 833491|
|   SEA| 711509|
|   IAH| 672528|
|   PHX| 661849|
|   LAS| 648466|
+------+-------+
only showing top 10 rows



O aeroporto com mais partidas é Aeroporto Internacional de Atlanta Hartsfield-Jackson.

### Número de voos pelo top 10 de destinos - 'Dest'

In [53]:
df_flights_clean.groupBy('Dest').count().orderBy(desc('count')).show(10)

+----+-------+
|Dest|  count|
+----+-------+
| ATL|1347297|
| ORD|1325509|
| DEN|1136193|
| DFW|1062170|
| CLT| 879096|
| LAX| 834181|
| SEA| 710775|
| IAH| 671574|
| PHX| 661392|
| LAS| 648924|
+----+-------+
only showing top 10 rows



O aeroporto com mais chegadas é Aeroporto Internacional de Atlanta Hartsfield-Jackson. Salientamos que ainda que os valores variem um pouco, os aeroportos mais relevantes nas partidas e chegadas são os mesmos, o que faz sentido, considerando que as rotas já estão estabelecidas de e para estes aeroportos.

## -----

### Atrasos na partidas - 'DepDel15'

É de notar que a variável DepDel15, apenas assume 2 valores - 0 e 1. Sendo que 1, corresponde a voos com atraso na partida, ou seja, voos atrasados por 15 ou mais minutos e 0, se os voos tiverem partido com menos de 15 minutos de atraso. 

In [54]:
df_flights_clean.groupBy("DepDel15").agg(F.count("*").alias("Number of flights")).show()

+--------+-----------------+
|DepDel15|Number of flights|
+--------+-----------------+
|     0.0|         23458242|
|     1.0|          4881267|
+--------+-----------------+



Há mais voos não atrasados na partida, do que o contrário.

### Top 10 companhias aéreas com atrasos nas partidas - 'DepDel15' e 'Airline'

In [59]:
df_flights_clean.select("DepDel15", "Airline").where("DepDel15=1").groupBy("Airline").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+--------------------+-----------------+
|             Airline|Number of flights|
+--------------------+-----------------+
|Southwest Airline...|          1131161|
|American Airlines...|           540742|
|SkyWest Airlines ...|           457436|
|Delta Air Lines Inc.|           419022|
|United Air Lines ...|           399404|
|     JetBlue Airways|           273833|
|   Republic Airlines|           171790|
|           Envoy Air|           151474|
|    Spirit Air Lines|           149826|
|         Comair Inc.|           149711|
+--------------------+-----------------+



É de notar que o top 10 de companhias aéreas com mais atrasos nas partidas são praticamente as mesmas que correspondem ao top 10  de aeroportos com mais voos, a única exceção é a companhia aérea Spirit Air Lines. A com mais atrasos é a Southwest Airlines.

### Atrasos nas partidas por ano - 'DepDel15' e 'Year'

In [61]:
df_flights_clean.select("DepDel15", "Year").where("DepDel15=1").groupBy("Year").agg(F.count("*").alias("Number of flights")).show()

+----+-----------------+
|Year|Number of flights|
+----+-----------------+
|2018|          1044682|
|2019|          1477915|
|2020|           431062|
|2021|          1068103|
|2022|           859505|
+----+-----------------+



Os atrasos nas partidas foram mais sentidos em 2019 e menos em 2020, devido à pandemia e a todas as restrições a ela associadas. Salientamos que não temos os dados completos para o ano de 2022, o que também vai impactar os resultados associados ao 'Quarter', 'Month', 'DayofMonth' e 'DayofWeek'.

### Atrasos nas partidas por trimestre - 'DepDel15' e 'Quarter'

In [67]:
df_flights_clean.select("DepDel15", "Quarter").where("DepDel15=1").groupBy("Quarter").agg(F.count("*").alias("Number of flights")).sort('Quarter').show()

+-------+-----------------+
|Quarter|Number of flights|
+-------+-----------------+
|      1|          1216098|
|      2|          1320365|
|      3|          1254130|
|      4|          1090674|
+-------+-----------------+



Os atrasos nas partidas são mais sentidos no segundo trimestre e menos no quarto. Os atrasos no primeiro e terceiro trimestres são parecidos.

### Atrasos nas partidas por mês - 'DepDel15' e 'Month'

In [69]:
df_flights_clean.select("DepDel15", "Month").where("DepDel15=1").groupBy("Month").agg(F.count("*").alias("Number of flights")).sort('Month').show()

+-----+-----------------+
|Month|Number of flights|
+-----+-----------------+
|    1|           413990|
|    2|           401130|
|    3|           400978|
|    4|           383490|
|    5|           409288|
|    6|           527587|
|    7|           551043|
|    8|           417018|
|    9|           286069|
|   10|           348365|
|   11|           322306|
|   12|           420003|
+-----+-----------------+



O mês com mais atrasos nas partidas é julho - 7 - e o com menos é novembro - 11.

### Atrasos nas partidas por dia do mês - 'DepDel15' e 'DayofMonth'

In [73]:
df_flights_clean.select("DepDel15", "DayofMonth").where("DepDel15=1").groupBy("DayofMonth").agg(F.count("*").alias("Number of flights")).sort('DayofMonth').show()

+----------+-----------------+
|DayofMonth|Number of flights|
+----------+-----------------+
|         1|           163376|
|         2|           163094|
|         3|           158438|
|         4|           140853|
|         5|           141602|
|         6|           155992|
|         7|           158927|
|         8|           158102|
|         9|           161175|
|        10|           159534|
|        11|           165785|
|        12|           169513|
|        13|           171745|
|        14|           160419|
|        15|           164919|
|        16|           162939|
|        17|           164926|
|        18|           177675|
|        19|           171931|
|        20|           171953|
+----------+-----------------+
only showing top 20 rows



Dos dias visíveis, o com mais atrasos é o dia 20 e o com menos é o dia 4. Mais uma vez, colocamos a hipótese de que o dia com menos atrasos é o dia 31, porque é um dia que ocorre em todos os meses. Esta vai ser corroborada no gráfico 'Número de voos atrasados na partida por dia do mês' no notebook 'Visualização'.

### Atrasos nas partidas por dia da semana - 'DepDel15' e 'DayofWeek'

In [76]:
df_flights_clean.select("DepDel15", "DayofWeek").where("DepDel15=1").groupBy("DayofWeek").agg(F.count("*").alias("Number of flights")).sort('DayofWeek').show()

+---------+-----------------+
|DayofWeek|Number of flights|
+---------+-----------------+
|        1|           748214|
|        2|           616739|
|        3|           625170|
|        4|           761820|
|        5|           794939|
|        6|           595880|
|        7|           738505|
+---------+-----------------+



O dia da semana com mais atrasos nas partidas é quinta e o com menos é sábado.

### Top 10 de atrasos nas partidas por origem - 'DepDel15' e 'Origin'

In [78]:
df_flights_clean.select("DepDel15", "Origin").where("DepDel15=1").groupBy("Origin").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+------+-----------------+
|Origin|Number of flights|
+------+-----------------+
|   ORD|           248917|
|   DEN|           241926|
|   DFW|           211241|
|   ATL|           207956|
|   CLT|           139740|
|   LAX|           139398|
|   EWR|           133151|
|   LAS|           131881|
|   PHX|           120779|
|   MCO|           119677|
+------+-----------------+



Os voos com mais atrasos nas partidas vêm do Aeroporto Internacional O'Hare, sendo que este é o segundo aeroporto com mais voos.

### Top 10 de atrasos nas partidas por destino - 'DepDel15' e 'Dest'

In [15]:
df_flights_clean.select("DepDel15", "Dest").where("DepDel15=1").groupBy("Dest").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+----+-----------------+
|Dest|Number of flights|
+----+-----------------+
| ORD|           223344|
| DEN|           188175|
| DFW|           178987|
| ATL|           175550|
| LAX|           136812|
| EWR|           136637|
| CLT|           127284|
| LAS|           122632|
| SFO|           116644|
| MCO|           113850|
+----+-----------------+



Os voos com mais atrasos nas partidas vão para o Aeroporto Internacional O'Hare.

### Diversidade de estatísticas associadas aos voos de partida atrasados

In [24]:
# número total de voos
total_flights = df_flights_clean.count()

# voos atrasados mais de 15 min
delayed_flights = df_flights_clean.filter(F.col("DepDel15") == 1).count()

# voos a tempo
delayed_flights1 = df_flights_clean.filter(F.col("DepDel15") == 0).count()

# % voos atrasados
percent_delayed = round((delayed_flights / total_flights) * 100, 2)

# % voos a tempo
percent_on_time = round((delayed_flights1 / total_flights) * 100, 2)

table = [(total_flights, delayed_flights, percent_delayed, delayed_flights1, percent_on_time)]

df_table = spark.createDataFrame(table, ["Número total de voos", "Voos atrasados na partida", "% Voos atrasados na partida", "Voos a tempo na partida", "% Voos a tempo"])

df_table.show()

+--------------------+-------------------------+---------------------------+-----------------------+--------------+
|Número total de voos|Voos atrasados na partida|% Voos atrasados na partida|Voos a tempo na partida|% Voos a tempo|
+--------------------+-------------------------+---------------------------+-----------------------+--------------+
|            28339509|                  4881267|                      17.22|               23458242|         82.78|
+--------------------+-------------------------+---------------------------+-----------------------+--------------+



De um total de 28339509 voos, 4881267 são atrasados na partida, o que representa 17.22% de todos os voos e os a tempo são  23458242, o que corresponde a 82.78% .

### Atrasos na chegadas - 'ArrDel15'

É de notar que a variável ArrDel15, apenas assume 2 valores - 0 e 1. Sendo que 1, corresponde a voos com atraso na chegada, ou seja, voos atrasados por 15 ou mais minutos e 0, se os voos tiverem chegado com menos de 15 minutos de atraso. 

In [83]:
df_flights_clean.groupBy("ArrDel15").agg(F.count("*").alias("Number of flights")).show()

+--------+-----------------+
|ArrDel15|Number of flights|
+--------+-----------------+
|     0.0|         23334258|
|     1.0|          5005251|
+--------+-----------------+



Há mais voos sem atrasos nas chegadas do que o contrário.

In [84]:
df_arrdel15 = df_flights_clean.filter(df_flights_clean.ArrDel15 == 1)

### Top 10 companhias aéreas com atrasos nas chegadas

In [86]:
df_arrdel15.groupBy("Airline").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+--------------------+-----------------+
|             Airline|Number of flights|
+--------------------+-----------------+
|Southwest Airline...|           958557|
|American Airlines...|           565689|
|SkyWest Airlines ...|           501389|
|United Air Lines ...|           426434|
|Delta Air Lines Inc.|           423125|
|     JetBlue Airways|           274876|
|   Republic Airlines|           203657|
|           Envoy Air|           179823|
|         Comair Inc.|           162124|
|    Spirit Air Lines|           155972|
+--------------------+-----------------+



A companhia aérea com mais atrasos nas chegadas é Southwest Airlines. Podemos observarv que o top 10 de companhias aéreas com mais atrasos nas partidas são praticamente as mesmas que correspondem ao top 10  de aeroportos com mais voos, a única exceção é a companhia aérea Spirit Air Line.

### Atrasos nas chegadas por ano

In [88]:
df_arrdel15.groupBy("Year").agg(F.count("*").alias("Number of flights")).show()

+----+-----------------+
|Year|Number of flights|
+----+-----------------+
|2018|          1086881|
|2019|          1527454|
|2020|           468896|
|2021|          1068058|
|2022|           853962|
+----+-----------------+



O ano com mais atrasos nas chegadas é 2019 e o com menos é 2022, considerando o fecho de fronteiras, o que fez com que circulassem menos voos. Relativamente a 2022 só temos dados até julho, o que vai afetar os resultados associados a 'Year', 'Quarter', 'Month', 'DayofMonth' e 'DayofWeek'. 

### Atrasos nas chegadas por trimestre

In [89]:
df_arrdel15.groupBy("Quarter").agg(F.count("*").alias("Number of flights")).sort('Quarter').show()

+-------+-----------------+
|Quarter|Number of flights|
+-------+-----------------+
|      1|          1260276|
|      2|          1329698|
|      3|          1285847|
|      4|          1129430|
+-------+-----------------+



O trimestre com mais atrasos nas chegadas é o segundo e o com menos é o quarto.

### Atrasos nas chegadas por mês

In [91]:
df_arrdel15.groupBy("Month").agg(F.count("*").alias("Number of flights")).sort('Month').show()

+-----+-----------------+
|Month|Number of flights|
+-----+-----------------+
|    1|           426310|
|    2|           426616|
|    3|           407350|
|    4|           388531|
|    5|           410491|
|    6|           530676|
|    7|           556411|
|    8|           432240|
|    9|           297196|
|   10|           364446|
|   11|           334469|
|   12|           430515|
+-----+-----------------+



O mês com mais atrasos é julho - 7 - e o com menos é setembro - 9.

### Atrasos nas chegadas por dia do mês

In [93]:
df_arrdel15.groupBy("DayofMonth").agg(F.count("*").alias("Number of flights")).sort('DayofMonth').show()

+----------+-----------------+
|DayofMonth|Number of flights|
+----------+-----------------+
|         1|           165609|
|         2|           166587|
|         3|           160485|
|         4|           141594|
|         5|           144130|
|         6|           162624|
|         7|           165503|
|         8|           162128|
|         9|           166086|
|        10|           164439|
|        11|           172700|
|        12|           176786|
|        13|           180683|
|        14|           166221|
|        15|           170036|
|        16|           166532|
|        17|           169099|
|        18|           181354|
|        19|           174817|
|        20|           175476|
+----------+-----------------+
only showing top 20 rows



Dos dias visíveis, o com mais atrasos de chegada é o dia 18 e o com menos é o dia 5. Mais uma vez, colocamos a hipótese de que o dia com menos atrasos é o dia 31, porque é um dia que ocorre em todos os meses. Esta vai ser corroborada no gráfico 'Número de voos atrasados na chegada por dia do mês' presente no notebook 'Visualização'.

### Atrasos nas chegadas por dia da semana

In [94]:
df_arrdel15.groupBy("DayofWeek").agg(F.count("*").alias("Number of flights")).sort('DayofWeek').show()

+---------+-----------------+
|DayofWeek|Number of flights|
+---------+-----------------+
|        1|           765874|
|        2|           632982|
|        3|           654294|
|        4|           798263|
|        5|           819020|
|        6|           590641|
|        7|           744177|
+---------+-----------------+



O dia da semana com menos atrasos de chegada é sábado e o com mais é segunda.

### Top 10 de atrasos nas chegadas por origem

In [96]:
df_arrdel15.groupBy("Origin").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+------+-----------------+
|Origin|Number of flights|
+------+-----------------+
|   ORD|           280139|
|   DEN|           239418|
|   DFW|           220143|
|   ATL|           196709|
|   CLT|           155455|
|   EWR|           137864|
|   LAX|           133915|
|   LAS|           122183|
|   IAH|           120175|
|   MCO|           116983|
+------+-----------------+



Os voos com mais atrasos nas chegadas vêm do Aeroporto Internacional O'Hare.

### Top 10 de atrasos nas chegadas por destino

In [95]:
df_arrdel15.groupBy("Dest").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+----+-----------------+
|Dest|Number of flights|
+----+-----------------+
| ORD|           244667|
| DEN|           201321|
| DFW|           200131|
| ATL|           176919|
| EWR|           149922|
| LAX|           140527|
| CLT|           127368|
| SFO|           119300|
| LAS|           118019|
| SEA|           117747|
+----+-----------------+



Os voos com mais atrasos nas chegadas vão para o Aeroporto Internacional O'Hare.

### Diversidade de estatísticas associadas aos voos de chegada atrasados

In [23]:
# número de voos
total_flights = df_flights_clean.count()

# número de voos atrasados mais de 15 min
delayed_flights = df_flights_clean.filter(F.col("ArrDel15") == 1).count()

# número de voos a tempo
delayed_flights1 = df_flights_clean.filter(F.col("ArrDel15") == 0).count()

# % voos atrsados
percent_delayed = round((delayed_flights / total_flights) * 100, 2)

# % voos a tempo
percent_on_time =  round((delayed_flights1 / total_flights) * 100, 2)

table = [(total_flights, delayed_flights, percent_delayed, delayed_flights1, percent_on_time)]

df_table = spark.createDataFrame(table, ["Número total de voos", "Voos atrasados à chegada", "% Voos atrasados à chegada", "Voos a tempo à chegada", "% Voos a tempo"])

df_table.show()

+--------------------+------------------------+--------------------------+----------------------+--------------+
|Número total de voos|Voos atrasados à chegada|% Voos atrasados à chegada|Voos a tempo à chegada|% Voos a tempo|
+--------------------+------------------------+--------------------------+----------------------+--------------+
|            28339509|                 5005251|                     17.66|              23334258|         82.34|
+--------------------+------------------------+--------------------------+----------------------+--------------+



De um total de 28339509 voos,  5005251 estão atrasados à chegada, o que representa 17.66% de todos os voos e os a tempo são 23334258, o que corresponde a 82.34% .

## -----

### Top 10 de número de voos e diferença de minutos entre a partida planeada e real - 'DepDelayMinutes'¶

In [17]:
df_flights_clean.groupBy("DepDelayMinutes").agg(F.count("*").alias("Number of flights")).sort(F.desc("Number of flights")).limit(10).show()

+---------------+-----------------+
|DepDelayMinutes|Number of flights|
+---------------+-----------------+
|            0.0|         19200162|
|            1.0|           584834|
|            2.0|           467851|
|            3.0|           411426|
|            4.0|           371837|
|            5.0|           341400|
|            6.0|           307252|
|            7.0|           283375|
|            8.0|           261189|
|            9.0|           242226|
+---------------+-----------------+



19200162 voos não têm qualquer atraso.

### Estatísticas sobre as companhias aéreas e aeroportos

In [15]:
# Número de companhias aéreas
unique_carriers = df_flights_clean.select("Airline").distinct().count()

# Número de aeroportos
unique_airports = df_flights_clean.select(F.concat(F.col("Origin"), F.lit("-"), F.col("Dest")).alias("Airport")).distinct().count()

table = [( unique_carriers, unique_airports)]

df_table = spark.createDataFrame(table, ["Unique Carriers", "Unique Airports"])

df_table.show()

+---------------+---------------+
|Unique Carriers|Unique Airports|
+---------------+---------------+
|             28|           8182|
+---------------+---------------+



Temos 28 companhias aéreas e 8182 aeroportos.

### Top 10 de rotas com mais atrasos

In [27]:
df_flights_clean \
    .groupBy("Origin", "Dest") \
    .agg(avg("DepDelayMinutes").alias("AvgDepDelayMinutes")) \
    .orderBy("AvgDepDelayMinutes", ascending=False) \
    .limit(10) \
    .show()

+------+----+------------------+
|Origin|Dest|AvgDepDelayMinutes|
+------+----+------------------+
|   RDM| MFR|            1443.0|
|   MDT| HPN|             759.0|
|   VPS| SRQ|             720.0|
|   DAB| DTW|             248.0|
|   SCE| CID|             246.0|
|   CHA| MSP|             201.0|
|   ICT| DAY|             181.0|
|   ERI| ITH|             156.0|
|   DSM| PIA|             155.0|
|   JAX| TPA|             144.0|
+------+----+------------------+



A rota com mais atraso é do Redmond Municipal Airport para o Rogue Valley International Medford Airport.
